In [26]:
import pandas as pd
import os
import numpy as np

%reload_ext autoreload
%autoreload 2

## Residential (SF)

#### Create df with hourly LADWP peak pricing periods (low peak, high peak, base) in 2019. 

##### Note: The year 2019 starts on a Tuesday. If running a different year of Harbor data, need to adjust pricing periods to start on the correct day of the week.

In [27]:
#Constructed this csv based on LADWP retail rate pricing periods for each day and hour of the week. 
peak_periods = pd.read_csv('data/peak_periods_2019.csv')

In [28]:
#Expand to full year.
almost_full_year = pd.concat([peak_periods]*52, ignore_index=True)
last_hours = peak_periods[0:24]
full_year = pd.concat([almost_full_year, last_hours], ignore_index=True)

In [29]:
#Label each hour in the year with its corresponding seasonal pricing period (by month).

jan_march_days = 90
jan_march_hour_start = 0
jan_march_hour_end = jan_march_days*24-1

apr_may_days = 61
apr_may_hour_start = jan_march_hour_end + 1
apr_may_hour_end = apr_may_hour_start + (apr_may_days*24) -1

jun_days = 30
jun_hour_start = apr_may_hour_end + 1
jun_hour_end = jun_hour_start + (jun_days*24) -1

jul_sept_days = 92
jul_sept_hour_start = jun_hour_end + 1
jul_sept_hour_end = jul_sept_hour_start + (jul_sept_days*24) -1

oct_dec_days = 92
oct_dec_hour_start = jul_sept_hour_end + 1
oct_dec_hour_end = oct_dec_hour_start + (oct_dec_days*24) -1

full_year.loc[jan_march_hour_start:jan_march_hour_end,'month'] = 'jan_mar'
full_year.loc[apr_may_hour_start:apr_may_hour_end,'month'] = 'apr_may'
full_year.loc[jun_hour_start:jun_hour_end,'month'] = 'jun'
full_year.loc[jul_sept_hour_start:jul_sept_hour_end,'month'] = 'jul_sept'
full_year.loc[oct_dec_hour_start:oct_dec_hour_end,'month'] = 'oct_dec'

#### Read in LADWP residential retail TOU rate table and assign rates to each hour in year 2019.

In [30]:
res_tou_rate_lookup = pd.read_csv('data/res_retail_tou_rate_period_lookup.csv')

In [31]:
month_period = full_year.groupby(['month', 'period'])

In [32]:
res_hourly_tou_rate = pd.DataFrame()

for name, group in month_period:
    month = name[0]
    period = name[1]
    month_inds = res_tou_rate_lookup['month'] == month
    rate = res_tou_rate_lookup.loc[month_inds, period].item()
    #Convert 2021$ to 2018$ (0.96/1) and mulitply by 1000 to get price per MWh.
    group['rate_sf'] = rate * 0.96 * 1000

    res_hourly_tou_rate = pd.concat([res_hourly_tou_rate, group])
    
res_hourly_tou_rate['units'] = '2018$/MWh'
res_hourly_tou_rate = res_hourly_tou_rate.sort_index()

In [34]:
res_hourly_tou_rate.to_csv('data/ladwp_res_hourly_tou_rate.csv')

## Commercial & Residential (MF)

#### Get hourly pricing period file from above and adjust peak pricing periods for commercial and residential MF sector.

In [86]:
jan_may = ['jan_mar','apr_may']
jan_may_inds = full_year['month'].isin(jan_may)
full_year.loc[jan_may_inds, 'month_comm'] = 'jan_may'

jun_sept = ['jun','jul_sept']
jun_sept_inds = full_year['month'].isin(jun_sept)
full_year.loc[jun_sept_inds, 'month_comm'] = 'jun_sept'

oct_dec_inds = full_year['month']=='oct_dec'
full_year.loc[oct_dec_inds, 'month_comm'] = 'oct_dec'

In [87]:
month_period_comm = full_year.groupby(['month_comm', 'period'])

#### Create hourly rate profile for commercial and res mf and save csv.

##### Read in commercial TOU rate.

In [88]:
comm_tou_rate_lookup = pd.read_csv('data/comm_retail_rate_tou_lookup.csv')

##### Construct hourly rate profile for rates A-1, A-2, and R-3.

In [89]:
a2_inds = comm_tou_rate_lookup['rate']=='A2'
jan_may_inds = comm_tou_rate_lookup['month']=='jan_may'
jun_sept_inds = comm_tou_rate_lookup['month']=='jun_sept'
oct_dec_inds = comm_tou_rate_lookup['month']=='oct_dec'

In [90]:
comm_hourly_tou_rate = pd.DataFrame()

for name, group in month_period_comm:
    month_comm = name[0]
    period = name[1]
    month_comm_inds = comm_tou_rate_lookup['month'] == month_comm
    
    small_comm_rate_inds = comm_tou_rate_lookup['rate'] == 'A1'
    large_comm_rate_inds = comm_tou_rate_lookup['rate'] == 'A2'
    mf_rate_inds = comm_tou_rate_lookup['rate'] == 'R3'
    
    small_comm_rate = comm_tou_rate_lookup.loc[month_comm_inds & small_comm_rate_inds, period].item()
    large_comm_rate = comm_tou_rate_lookup.loc[month_comm_inds & large_comm_rate_inds, period].item()
    mf_rate = comm_tou_rate_lookup.loc[month_comm_inds & mf_rate_inds, period].item()
    
    #Convert 2021$ to 2018$ (0.96/1) and mulitply by 1000 to get price per MWh or MW.
    group['rate_small_comm'] = small_comm_rate * 0.96 * 1000
    group['rate_large_comm'] = large_comm_rate * 0.96 * 1000
    group['rate_mf'] = mf_rate * 0.96 * 1000

    comm_hourly_tou_rate = pd.concat([comm_hourly_tou_rate, group])
    
comm_hourly_tou_rate['units'] = '2018$/MWh'
comm_hourly_tou_rate = comm_hourly_tou_rate.sort_index()

In [91]:
comm_hourly_tou_rate

,day,hour_starting,period,month,month_comm,rate_small_comm,rate_large_comm,rate_mf,units
0,tues,0,base,jan_mar,jan_may,67.9872,37.3920,44.5248,2018$/MWh
1,tues,1,base,jan_mar,jan_may,67.9872,37.3920,44.5248,2018$/MWh
2,tues,2,base,jan_mar,jan_may,67.9872,37.3920,44.5248,2018$/MWh
3,tues,3,base,jan_mar,jan_may,67.9872,37.3920,44.5248,2018$/MWh
4,tues,4,base,jan_mar,jan_may,67.9872,37.3920,44.5248,2018$/MWh
...,...,...,...,...,...,...,...,...,...
8755,tues,19,low,oct_dec,oct_dec,96.0000,54.6048,44.5248,2018$/MWh
8756,tues,20,base,oct_dec,oct_dec,67.9872,37.3920,44.5248,2018$/MWh
8757,tues,21,base,oct_dec,oct_dec,67.9872,37.3920,44.5248,2018$/MWh
8758,tues,22,base,oct_dec,oct_dec,67.9872,37.3920,44.5248,2018$/MWh


In [92]:
comm_hourly_tou_rate.to_csv('data/ladwp_comm_hourly_tou_rate.csv')

#### Calculate annual demand charge savings for commercial and res mf EE measures and save csv.

##### Read in file with 1) resource ratios of demand charge savings to capacity and 2) demand charge rates based on monthly peak.

In [125]:
demand_charge_rates = pd.read_csv('data/demand_charge_monthly_peak_ratios.csv')

In [126]:
#Add in demand charge rate based on yearly peak (incurred monthly), which is the same for A-1, A-2, and R-3 rates. Convert from $2021 to $2018.
demand_charge_rates['yearly_peak_demand_charge_per_mw'] = 5.36 * 0.96 * 1000

comm_demand_charge_rates = demand_charge_rates[['hvac', 'refrigeration','comm_high_peak_demand_charge_per_mw',
       'comm_low_peak_demand_charge_per_mw', 'yearly_peak_demand_charge_per_mw']]

In [127]:
mf_demand_charge_rates = demand_charge_rates[['lighting', 'hvac', 'refrigeration','mf_demand_charge_per_mw', 'yearly_peak_demand_charge_per_mw']]

In [128]:
#Calculate res MF demand charge savings in each month based on yearly peak (for lighting, hvac, refrigeration). Yearly peak savings as a fraction of capacity are represented by the peak ratio in the hvac, refrig, and lighting columns.
mf_demand_charge_rates['lighting_yearly_peak_demand_charge_mw'] = 1*mf_demand_charge_rates['yearly_peak_demand_charge_per_mw']
mf_demand_charge_rates['hvac_yearly_peak_demand_charge_mw'] = 0.5*mf_demand_charge_rates['yearly_peak_demand_charge_per_mw']
mf_demand_charge_rates['refrigeration_yearly_peak_demand_charge_mw'] = 0.5*mf_demand_charge_rates['yearly_peak_demand_charge_per_mw']

<ipython-input-128-be461562e560>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf_demand_charge_rates['lighting_yearly_peak_demand_charge_mw'] = 1*mf_demand_charge_rates['yearly_peak_demand_charge_per_mw']


In [129]:
#Calculate demand charge savings in each month based on monthly peak (refrig, hvac, lighting).
mf_demand_charge_rates['lighting_monthly_peak_demand_charge_mw'] = mf_demand_charge_rates['lighting']*mf_demand_charge_rates['mf_demand_charge_per_mw']
mf_demand_charge_rates['refrigeration_monthly_peak_demand_charge_mw'] = mf_demand_charge_rates['refrigeration']*mf_demand_charge_rates['mf_demand_charge_per_mw']
mf_demand_charge_rates['hvac_monthly_peak_demand_charge_mw'] = mf_demand_charge_rates['hvac']*mf_demand_charge_rates['mf_demand_charge_per_mw']


In [130]:
#Calculate demand charge savings in each month based on yearly peak (for lighting, hvac, refrigeration). Yearly peak savings as a fraction of capacity are represented by the peak ratio in the hvac and refrig columns.
comm_demand_charge_rates['hvac_yearly_peak_demand_charge_mw'] = 0.5*comm_demand_charge_rates['yearly_peak_demand_charge_per_mw']
comm_demand_charge_rates['refrigeration_yearly_peak_demand_charge_mw'] = 0.5*comm_demand_charge_rates['yearly_peak_demand_charge_per_mw']


<ipython-input-130-72d5a74cb17f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comm_demand_charge_rates['hvac_yearly_peak_demand_charge_mw'] = 0.5*comm_demand_charge_rates['yearly_peak_demand_charge_per_mw']


In [131]:
#Calculate demand charge savings in each month based on monthly high peak (comm refrig and hvac).
comm_demand_charge_rates['refrigeration_high_peak_demand_charge_mw'] = comm_demand_charge_rates['refrigeration']*comm_demand_charge_rates['comm_high_peak_demand_charge_per_mw']
comm_demand_charge_rates['hvac_high_peak_demand_charge_mw'] = comm_demand_charge_rates['hvac']*comm_demand_charge_rates['comm_high_peak_demand_charge_per_mw']


In [132]:
#Calculate demand charge savings in each month based on monthly low peak (for comm refrig and hvac).
comm_demand_charge_rates['refrigeration_low_peak_demand_charge_mw'] = comm_demand_charge_rates['refrigeration']*comm_demand_charge_rates['comm_low_peak_demand_charge_per_mw']
comm_demand_charge_rates['hvac_low_peak_demand_charge_mw'] = comm_demand_charge_rates['hvac']*comm_demand_charge_rates['comm_low_peak_demand_charge_per_mw']

In [133]:
comm_demand_charge_savings = comm_demand_charge_rates.sum().reset_index()

In [135]:
comm_demand_charge_savings = comm_demand_charge_savings.iloc[5:]

In [137]:
comm_demand_charge_savings.rename(columns={'index':'resource_peakperiod', 0:'demand_charge_savings_per_mw'},inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [138]:
comm_demand_charge_savings.to_csv('data/comm_annual_demand_charge_savings.csv')

In [140]:
mf_demand_charge_savings = mf_demand_charge_rates.sum().reset_index()

In [143]:
mf_demand_charge_savings = mf_demand_charge_savings.iloc[5:]

In [144]:
mf_demand_charge_savings.rename(columns={'index':'resource_peakperiod', 0:'demand_charge_savings_per_mw'},inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [145]:
mf_demand_charge_savings

,resource_peakperiod,demand_charge_savings_per_mw
5,lighting_yearly_peak_demand_charge_mw,61747.2
6,hvac_yearly_peak_demand_charge_mw,30873.6
7,refrigeration_yearly_peak_demand_charge_mw,30873.6
8,lighting_monthly_peak_demand_charge_mw,90400.0
9,refrigeration_monthly_peak_demand_charge_mw,45200.0
10,hvac_monthly_peak_demand_charge_mw,30989.0


In [146]:
mf_demand_charge_savings.to_csv('data/mf_annual_demand_charge_savings.csv')